# Creating a Collaborative Closing Transaction

In this section, we'll build a lightning channel closing transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed, signed, and the messages exchanged between peers to coordinate the close. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the Chapter 1 – Channel Funding Transaction, where the funding transaction was created and confirmed on our regtest blockchain.

In [1]:
# run notebook
%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 1 - channel funding transaction/funding-transaction.ipynb"

2025-09-08T17:27:06.579000Z TestFramework (INFO): PRNG seed is: 3701658290322657702
2025-09-08T17:27:06.580000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test__l4x214d
Alice funding pubkey: 93b676043fabdb9a66327449fba5bebbc825995ae4c7901b85e39f704cf6f47e
Alice funding privkey: edee0fe4b1924802a1bb040209562366049af1c05141a6ca92d40bb1ff5add2b
Alice funding address: bcrt1pjwm8vppl40de5e3jw3ylhfd7h0yztx26unreqxu9uw0hqn8k73lq66w9e4
Bob funding pubkey: 3cbd1f8b15ad007aed22b960ca13612b6aa7e68bca895ebc88c3d07b29e24be3
Bob funding privkey: f777ba512d3097bc7ffe75324760b83f5e696ba3eb2c8ae0761cd4cc73b62943
Bob funding address: bcrt1p8j73lzc445q84mfzh9sv5ymp9d420e5te2y4a0ygc0g8k20zf03sg3u87f
Transaction creating Alice UTXO: 04594c89d11bd7c9849b8da3328d35c881a41df0302dcf1f8978c977b2510905
{
  "txid": "04594c89d11bd7c9849b8da3328d35c881a41df0302dcf1f8978c977b2510905",
  "hash": "16b41778541babd055b2bcc750cb5a932bcf10a6f26d97fa377469ee027b3655",
  "version": 2,
  "size": 289,

## Legacy cooperative closure

Compared to the base segwit v0 channel type, for simple taproot channels, the co-op close transaction now always signals RBF. In other words, the sequence field of the sole input to the cooperative close transaction MUST be less-than-or-equal to 0xfffffffd. This enables a future cooperative closure flow to support increasing the fee of subsequent close offers via RBF.

In addition, rather than adopt the existing cooperative closure fee rate "negotiation", the responder SHOULD now always accept the offer sent by the initiator. In other words, the cooperative close process now terminates after exactly 1 RTTs: initiator sends sigs with offer, with the responder echo'ing back the same fee rate. This serves to ensure that the co-op close process always terminates deterministically, and also plays nicer with the nonces: only a single message is ever signed by both sides for a coop close workflow.

    +-----------+                                        +---------+
    |           |--(1)-------- shutdown (nonce) -------->|         |
    |           |<-(2)-------- shutdown (nonce) ---------|         |
    |           |                                        |         |
    |   Alice   |--(3)-- closing_signed (partial sig) -->|   Bob   |
    |           |<-(4)-- closing_signed (partial sig) ---|         |
    |           |                                        |         |
    |           |                                        |         |
    |           |                                        |         |
    +-----------+                                        +---------+

## Modern RBF-based cooperative close

For taproot channels supporting RBF cooperative close, nonces are delivered just-in-time (JIT) with signatures using an asymmetric pattern:

Initial Exchange: Nonces are exchanged in shutdown messages

Each party sends their "closee nonce" - the nonce they'll use when signing as the closee. This nonce is used by the remote party when they act as closer.

#### RBF Iterations: Subsequent nonces are delivered with signatures

* closing_complete (from closer): uses partial_sig_w_nonce bundles the partial signature with the sender's closer nonce. The bundling is necessary because the closee doesn't know this nonce yet.

* closing_sig (from closee): uses partial_sig plus nonce (NextCloseeNonce) field

Asymmetric Roles:

* Closer: The party sending closing_complete, proposing a new fee
* Closee: The party sending closing_sig, accepting the fee proposal

Each party alternates between these roles during RBF iterations.

    +-----------+                                                    +---------+
    |           |--(1)-------------- shutdown (nonce) -------------->|         |
    |           |<-(2)-------------- shutdown (nonce) ---------------|         |
    |           |                                                    |         |
    |   Alice   |--(3)- closing_complete (partial sig & jit nonce) ->|   Bob   |
    |           |<-(4)------ closing_sig (partial sig & nonce)-------|         |
    |           |                                                    |         |
    |           |<-(5)- closing_complete (partial sig & jit nonce) ->|         |
    |           |--(6)------ closing_sig (partial sig) (nonce)------>|         |
    |           |                                                    |         |
    +-----------+                                                    +---------+



## The Unsined Transaxaction

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# Use `0xfffffffd` to enable a future rbf cooperative closure round
sequence = bytes.fromhex("fdff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Back to Alice
# 0.005 BTC discounted the fees
# In the legacy cooperative closure the channel founder pay the fees
# In the new rbf cooperative closure the closer pay the fees
output1_value_sat = int(float("0.005") * 100000000) - tx_fee_sat
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output1_spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey_b) + alice_funding_pubkey_b

# OUTPUT 2 - Back to Bob
# 0.005 BTC
output2_value_sat = int(float("0.005") * 100000000)
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output2_spk = bytes.fromhex("51") + varint_len(bob_funding_pubkey_b) + bob_funding_pubkey_b

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  020000000147c2a1b6391d26e4a8d5b5a582f7e106e1e8f355bd44405ee27fcdcc538125ea0000000000fdffffff02f49f07000000000022512093b676043fabdb9a66327449fba5bebbc825995ae4c7901b85e39f704cf6f47e20a10700000000002251203cbd1f8b15ad007aed22b960ca13612b6aa7e68bca895ebc88c3d07b29e24be300000000
{
  "txid": "94b4371c87876f1fc7495528834e56959232684e3ffe5a7a88118b74a9b79678",
  "hash": "94b4371c87876f1fc7495528834e56959232684e3ffe5a7a88118b74a9b79678",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "ea258153cccd7fe25e4044bd55f3e8e106e1f782a5b5d5a8e4261d39b6a1c247",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967293
    }
  ],
  "vout": [
    {
      "value": "0.00499700",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 93b676043fabdb9a66327449fba5bebbc825995ae4c7901b85e39f704cf6f47e",
        "desc": "rawtr(93b676043fabdb9a66327449fba5bebbc825995ae4c7901b85e39f70

## The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [3]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amounts = channel_value
sha_amounts = sha256(input_amounts)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) 

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)


## Signing the sighash

In [4]:
# Build participants to sort them using the same rule used in pubkeys aggregation.
participants = [
    (pubkey_alice_musig2, privkey_alice_musig2.get_bytes()),
    (pubkey_bob_musig2,   privkey_bob_musig2.get_bytes()),
]

# Reorder participants to match the sorted pubkeys
pk_to_tuple = {pk: (pk, sk) for pk, sk in participants}
participants = [pk_to_tuple[pk] for pk in sorted_pubkeys]

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk in participants:
    sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# Alice and Bob has exchanged already the pubnonces, so they can agregate them and create the session context
agg_nonce = nonce_agg(pubnonces)
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produces their partial signatures
# Partial signatures in the SAME order as pubkeys/participants
psigs = [sign(sec, sk, session) for sec, (_, sk) in zip(secnonces, participants)]

# Each side verify the other’s partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
agg_sig = partial_sig_agg(psigs, session)

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)


aggregated Schnorr verifies? True


## The signed transaction

In [5]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

coopclose_txid = node.sendrawtransaction(signed_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("coopclose_txid: ", coopclose_txid)

signed tx:  0200000000010147c2a1b6391d26e4a8d5b5a582f7e106e1e8f355bd44405ee27fcdcc538125ea0000000000fdffffff02f49f07000000000022512093b676043fabdb9a66327449fba5bebbc825995ae4c7901b85e39f704cf6f47e20a10700000000002251203cbd1f8b15ad007aed22b960ca13612b6aa7e68bca895ebc88c3d07b29e24be30140254a82c7ea70b7fbccab793169babb266cf1b9e480b5984d05bd6db87fa34c28934888e12bc268070e54cfc99a2ca105fc29e1439e6ba59ebde13e88df38ce7c00000000
{
  "txid": "94b4371c87876f1fc7495528834e56959232684e3ffe5a7a88118b74a9b79678",
  "hash": "7c20ba1aeb78ac239aa1d6edd10e98110afb927eccfb8af43522669d7e35b2cf",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "ea258153cccd7fe25e4044bd55f3e8e106e1f782a5b5d5a8e4261d39b6a1c247",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "254a82c7ea70b7fbccab793169babb266cf1b9e480b5984d05bd6db87fa34c28934888e12bc268070e54cfc99a2ca105fc29e1439e6ba59ebde13e88df3